# Importing some packages

In [7]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from pathlib import Path

In [3]:
def convert_time_interval(input_interval):
    """
    Function to convert a time interval in hours, days, or minutes to minutes.

    Parameters:
        input_interval (str): The time interval as a string.
                              For example: '5H' for 5 hours, '3D' for 3 days, or '300T' for 300 minutes.

    Returns:
        int: The equivalent time interval in minutes.
    """
    num = int(input_interval[:-1])  # Get the number (ignore the last character 'H', 'D', or 'T').
    unit = input_interval[-1]       # Get the unit of time ('H' for hours, 'D' for days, or 'T' for minutes).

    if unit == 'H':
        return f"{int(num/10 * 60)}T", f"{num * 60}T"  # Convert hours to minutes.
    elif unit == 'D':
        return f"{int(num/10 * 24 * 60)}T", f"{num * 24 * 60}T"  # Convert days to minutes.
    elif unit == 'T':
        if num<=5:
            return f"1T", f"{num}T"  # For minutes, return the value directly.
        else:
            return f"{int(num/5)}T", f"{num}T"  # For minutes, return the value directly.
    else:
        raise ValueError("Invalid input interval. Please use 'H' for hours, 'D' for days, or 'T' for minutes.")

In [4]:
convert_time_interval('4H')

('24T', '240T')

In [5]:
def resample_data(df, granularity):
    resample = df.resample(granularity)

    df_resampled = pd.DataFrame()
    df_resampled["open"] = resample["open"].first()
    df_resampled["high"] = resample["high"].max()
    df_resampled["low"] = resample["low"].min()
    df_resampled["close"] = resample["close"].last()
    df_resampled["tick_volume"] = resample["tick_volume"].sum()

    return df_resampled

In [10]:
low_tf, high_tf = convert_time_interval('4H')
print(low_tf, high_tf)

path = Path(r"C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\portfolio/\PublicTradingBot/\data/\data_15m/\15-minutes_USDCAD.csv")
df_15m = pd.read_csv(path, index_col="time", parse_dates=True)
df_15m.head()

24T 240T


,open,high,low,close,tick_volume,low_time,high_time
time,,,,,,,
2015-01-02 10:00:00,1.16483,1.16555,1.16483,1.16548,803,2015-01-02 10:00:00,2015-01-02 10:09:00
2015-01-02 10:15:00,1.16547,1.16583,1.16540,1.16581,596,2015-01-02 10:18:00,2015-01-02 10:27:00
2015-01-02 10:30:00,1.16581,1.16587,1.16497,1.16527,683,2015-01-02 10:39:00,2015-01-02 10:30:00
2015-01-02 10:45:00,1.16524,1.16539,1.16479,1.16488,660,2015-01-02 10:57:00,2015-01-02 10:51:00
2015-01-02 11:00:00,1.16490,1.16519,1.16472,1.16497,857,2015-01-02 11:09:00,2015-01-02 11:06:00
